In [20]:
vantage_broker_url = "http://home.johanvansoest.nl"
vantage_broker_username = "node1-user"
vantage_broker_password = "node1-password"

vantage_broker_encryption = None
vantage_broker_port = 5000
vantage_broker_api_path = "/api"

In [22]:
# Setup client connection
from vantage6.client import Client
client = Client(vantage_broker_url, vantage_broker_port, vantage_broker_api_path)
client.authenticate(vantage_broker_username, vantage_broker_password)
client.setup_encryption(vantage_broker_encryption)

In [49]:
collaboration_list = client.collaboration.list()
organization_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[organization_index]['organizations']:
    organization_ids_.append(organization['id'])

In [51]:
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {}
}

task = client.post_task(
    name="RetrieveVariables",
    image="jaspersnel/v6-colnames-py",
    collaboration_id=collaboration_list[organization_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

In [65]:
import time
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
colnamesLists = resultObj['result']

colnames = [item for sublist in colnamesLists for item in sublist]
colCount = {}
for colname in colnames:
    if colname in colCount:
        colCount[colname] = colCount[colname] + 1
    else:
        colCount[colname] = 1
print(colCount)
#print("")
#print(resultObj['log'])   

{'PatientID': 3, 'age': 3, 'Clinical.T.Stage': 3, 'Clinical.N.Stage': 3, 'Clinical.M.Stage': 3, 'Overall.Stage': 3, 'Histology': 3, 'gender': 3, 'Survival.time': 3, 'deadstatus.event': 3}
